In [1]:
import os

os.environ["PUBSUB_PROJECT_ID"] = "my-project"
os.environ["PUBSUB_EMULATOR_HOST"] = "localhost:8080"

from pubsub_demo.utils.pubsub import GooglePubsubClient

pubsub = GooglePubsubClient(project_id="my-project")
